In [36]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# Constants
num_classes = 2  # Number of classes (faces and non-faces)
image_size = 64  # Size of input images (assumes square images)

# Load training and testing data
train_data = ImageDataGenerator(rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data = ImageDataGenerator(rescale=1.0 / 255)

train_set = train_data.flow_from_directory("D:\\CU\\face_rec\\Dataset\\Train", target_size=(image_size, image_size), batch_size=32,
                                           class_mode='categorical')
test_set = test_data.flow_from_directory("D:\\CU\\face_rec\\Dataset\\Test", target_size=(image_size, image_size), batch_size=32,
                                         class_mode='categorical')

# Build CNN model
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(image_size, image_size, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_set, epochs=10, validation_data=test_set)

# Save the model
model.save('face_recognition_model.h5')



Found 218 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


Epoch 1/10
7/7 [==============================] - 5s 622ms/step - loss: 0.6644 - accuracy: 0.5688 - val_loss: 0.6208 - val_accuracy: 0.5275
Epoch 2/10
7/7 [==============================] - 4s 555ms/step - loss: 0.4528 - accuracy: 0.8119 - val_loss: 0.0275 - val_accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 4s 565ms/step - loss: 0.2785 - accuracy: 0.8624 - val_loss: 0.1281 - val_accuracy: 0.9725
Epoch 4/10
7/7 [==============================] - 4s 593ms/step - loss: 0.0726 - accuracy: 0.9908 - val_loss: 0.0340 - val_accuracy: 0.9862
Epoch 5/10
7/7 [==============================] - 4s 560ms/step - loss: 0.0361 - accuracy: 0.9862 - val_loss: 0.0312 - val_accuracy: 0.9817
Epoch 6/10
7/7 [==============================] - 4s 654ms/step - loss: 0.0860 - accuracy: 0.9679 - val_loss: 0.1050 - val_accuracy: 0.9725
Epoch 7/10
7/7 [==============================] - 5s 717ms/step - loss: 0.0936 - accuracy: 0.9587 - val_loss: 0.0163 - val_accuracy: 1.0000
Epoch 8/10
7/7 [====

In [48]:
import numpy as np
import cv2
from keras.models import load_model
model = load_model('face_recognition_model.h5')
# Load and preprocess the input image
image_path = "D:\\CU\\face_rec\\Dataset\\Test\\Akash\\a55.png"
image = cv2.imread(image_path)
image = cv2.resize(image, (64, 64))  # Resize the image to match the input size of the model
image = image.astype('float32') / 255  # Normalize the image
image = np.expand_dims(image, axis=0)  # Add an extra dimension to match the model's input shape

# Perform the prediction
prediction = model.predict(image)
print(prediction)
class_index = np.argmax(prediction)
print(class_index)
class_label = train_set.class_indices  # Use the class indices from the training set

# Print the predicted class label
for label, index in class_label.items():
    if index == class_index:
        print("Predicted class:", label)
        break


1/1 [==============================] - 0s 74ms/step
[[0.9778016  0.02219838]]
0
Predicted class: Akash


In [50]:
import os
import cv2
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model("D:\\CU\\face_rec\\face_recognition_model.h5")

# Directory containing the images to test
image_directory = "D:\\CU\\face_rec\\Dataset\\Test"

# Iterate over all files in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read and preprocess the image
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))  # Resize if needed
        image = image / 255.0  # Normalize pixel values
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        # Perform inference
        predictions = model.predict(image)
        class_index = np.argmax(predictions[0])
        class_name = 'unknown'  # Replace with your own class names
        if class_index == 0:
            class_name = 'class_a'
        elif class_index == 1:
            class_name = 'class_b'
        # Add more elif statements for additional classes

        print(f"Image: {filename} - Predicted class: {class_name}")
